In [1]:
import cx_Oracle
import os
import pandas as pd
import xlwings as xw
import json
import datetime as dt
my_date_handler = lambda year, month, day, **kwargs: "%04i-%02i-%02i" % (year, month, day)
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()


In [71]:
connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()


In [4]:
sel_sht = wb_cy.selection.sheet
last_col = sel_sht.range('XFD9').end('left').column
last_row = sel_sht.range('A1048576').end('up').row

In [27]:
col_names = sel_sht.range((9,1),(9,last_col)).value
col_names
tb_rng = sel_sht.range((10,1),(last_row,last_col)).options(numbers=lambda e: str(int(e)),date=my_date_handler).value

In [28]:
df_bin = pd.DataFrame(tb_rng,columns=col_names)
df_bin['BIN_INDEX']=None
# df_bin.set_index('BIN_INDEX',inplace=True)

In [29]:
df_bin = df_bin.fillna('')
df_bin.set_index('BIN_INDEX',inplace=True)

In [30]:
df_bin

,ARTICLE_NUMBER,BIN,BIN_OLD,SUBINVENTORY
BIN_INDEX,,,,
,34-100390-002-9,"2C1804,2C1405",,KR_SERV01
,29128030,2B1801,,KR_SERV01
,29089545,"BULK,2C1405",,KR_SERV01
,29088185,"2C1604,2C1603",,KR_SERV01
,29088182,"2C1602,BULK,2C1603",,KR_SERV01
...,...,...,...,...
,18102926,2B1804,,KR_SERV01
,19781401,2B1803,,KR_SERV01
,29282854,2B1802,,KR_SERV01


In [33]:
def clear_form_bin():
    sel_sht = wb_cy.selection.sheet
    sel_sht.range("C2:C7").clear_contents()

    mode_cel = sel_sht.range("C2")
    mode_cel.value = 'CHANGE_BIN'

In [ ]:
def change_mode_bin():
    sel_sht = wb_cy.selection.sheet
    mode_cel = sel_sht.range("C2")
    if mode_cel.value == "CHANGE_BIN":
        mode_cel.value = 'REGIST_BIN'
    else : 
        mode_cel.value = 'CHANGE_BIN'


In [ ]:
wb_cy.app.alert("'REGIST_BIN' 모드에서는 파트번호,BIN 만 입력해도 등록이 가능합니다.")
wb_cy.app.alert("'CHANGE_BIN' 모드에서는 NEW_BIN에 들어가는 값이 새로운 BIN으로 등록되며 다른 값들은 적용되지 않습니다.")

In [36]:
is_protect = sel_sht.api.ProtectContents

False

In [59]:
# select_row
sel_sht = wb_cy.selection.sheet
last_col = sel_sht.range('XFD9').end('left').column
last_row = sel_sht.range('A1048576').end('up').row
sel_cells = sel_sht.range(wb_cy.selection.address)
col_num = sel_cells.column
if type(sel_cells.value) is list :
    wb_cy.app.alert("하나의 셀만 선택 후 진행해주세요. 두 개 이상은 불가합니다.","Change Cell WARNING")
#     return

In [60]:
import string

In [65]:
col_alphabet =string.ascii_uppercase[col_num-1]

In [69]:
off_set_rng = sel_cells.address.replace(col_alphabet,'A')
bin_idx = sel_sht.range(off_set_rng).options(numbers=lambda e : str(int(e))).value

In [92]:
df_bin_idx = pd.DataFrame(cursor.execute(f"select BIN_INDEX, ARTICLE_NUMBER, BIN, SUBINVENTORY from svc_bin where bin_index = '{bin_idx}'").fetchone())

In [99]:
sel_sht.range("C3").options(transpose=True).value = list(df_bin_idx[0])

In [100]:
def select_row_bin():
    sel_sht = wb_cy.selection.sheet
    last_col = sel_sht.range('XFD9').end('left').column
    last_row = sel_sht.range('A1048576').end('up').row
    sel_cells = sel_sht.range(wb_cy.selection.address)
    col_num = sel_cells.column
    bin_mode = sel_sht.range("C2").value
    col_alphabet =string.ascii_uppercase[col_num-1]
    
    if bin_mode != "CHANGE_BIN":
        wb_cy.app.alert("'CHANGE_BIN' 모드에서만 사용가능합니다.","Change Cell WARNING")
        return
    if type(sel_cells.value) is list :
        wb_cy.app.alert("하나의 셀만 선택 후 진행해주세요. 두 개 이상은 불가합니다.","Change Cell WARNING")
        return
    
    off_set_rng = sel_cells.address.replace(col_alphabet,'A')
    bin_idx = sel_sht.range(off_set_rng).options(numbers=lambda e : str(int(e))).value
    df_bin_idx = pd.DataFrame(cursor.execute(f"select BIN_INDEX, ARTICLE_NUMBER, BIN, SUBINVENTORY from svc_bin where bin_index = '{bin_idx}'").fetchone())
    sel_sht.range("C3").options(transpose=True).value = list(df_bin_idx[0])

In [116]:
def change_bin():
    sel_sht = wb_cy.selection.sheet
    bin_forms = sel_sht.range("C2:C7").options(numbers=lambda e : str(int(e))).value
    if bin_forms[-1] is None:
        wb_cy.app.alert("'NEW_BIN'에 값이 없습니다. 값을 입력 후 다시 시도해주세요.","Empty Cell")
    #     return
    cursor.execute(f"UPDATE SVC_BIN SET BIN = '{bin_forms[-1]}' WHERE BIN_INDEX = '{bin_forms[1]}'")
    cursor.execute("commit")

# clean form bring_data

In [188]:
# regist_bin()
def regist_bin():
    sel_sht = wb_cy.selection.sheet
    bin_mode = sel_sht.range("C2").value
    bin_forms = sel_sht.range("C2:C7").options(numbers=lambda e : str(int(e))).value
    last_col = sel_sht.range('XFD9').end('left').column
    last_row = sel_sht.range('A1048576').end('up').row
    col_names = sel_sht.range((9,1),(9,last_col)).value
    if bin_mode != "REGIST_BIN":
        wb_cy.app.alert("'REGIST_BIN' 모드에서만 사용가능합니다.","Regist Cell WARNING")
        return
    none_count = [bin_forms[2],bin_forms[-1]].count(None)
    if none_count > 0 :
        wb_cy.app.alert("'ARTICLE_NUMBER', 'NEW_BIN'에는 필수로 값이 있어야합니다.","Regist Cell WARNING")
        return

    is_db_data = not pd.DataFrame(cursor.execute(f"SELECT * FROM SVC_BIN WHERE ARTICLE_NUMBER = '{bin_forms[2]}'").fetchone()).empty
    if is_db_data is True:
        wb_cy.app.alert(f"입력하신 'ARTICLE_NUMBER' : {bin_forms[2]} 는 등록되어있습니다. 종료합니다.","Regist Cell WARNING")
        return

    df_regist = pd.DataFrame([bin_forms]).drop(columns=[0,3,5])
    df_regist[1]=None
    df_regist.insert(2,'BIN_OLD',None)
    df_regist.insert(2,'BIN',bin_forms[-1])
    df_regist.columns = col_names
    return df_regist

In [195]:
regist_bin()